In [1]:
import os
os.getcwd()


'C:\\Users\\Admin\\Desktop\\comment-toxicity-detection\\notebooks'

In [2]:
import pandas as pd

train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

train.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:

print(train.shape)
print(train.columns)

(159571, 8)
Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')


In [4]:
train.info()
train.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [6]:
train['comment_text'].str.len().describe()
train.iloc[:, 2:].sum().sort_values(ascending=False)


toxic            15294
obscene           8449
insult            7877
severe_toxic      1595
identity_hate     1405
threat             478
dtype: int64

In [7]:
import re
import string

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"\d+", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    return text


In [8]:
import sys
print(sys.executable)


C:\Users\Admin\anaconda3\python.exe


In [9]:
import tensorflow as tf
print(tf.__version__)


2.20.0


In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def tokenize_and_pad(texts, max_len=150):
    tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    padded = pad_sequences(sequences, maxlen=max_len, padding='post')
    return padded, tokenizer


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

def build_model(vocab_size, max_len, output_dim):
    model = Sequential([
        Embedding(vocab_size, 128),
        LSTM(64, return_sequences=False),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dense(output_dim, activation='sigmoid')
    ])
    
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return model


In [29]:
# Step 1 — Import src folder
import sys
sys.path.append("../src")

# Step 2 — Import libraries
import pandas as pd
from preprocessing import clean_text, tokenize_and_pad
from model import build_model

# Step 3 — Load dataset
train = pd.read_csv("../data/train.csv")

# Step 4 — Prepare data
X = train['comment_text'].apply(clean_text)
y = train.iloc[:, 2:]

# Step 5 — Tokenize text
X_pad, tokenizer = tokenize_and_pad(X)

# Step 6 — Build model
def build_model(vocab_size, max_len, output_dim):
    model = Sequential([
        Embedding(vocab_size, 128),
        LSTM(128, return_sequences=True),
        Dropout(0.3),
        LSTM(64),
        Dense(64, activation='relu'),
        Dense(output_dim, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

# Step 7 — Train model
model.fit(
    X_pad,
    y,
    epochs=7,
    batch_size=85,
    validation_split=0.5
)

# Step 8 — Save model
import os
os.makedirs("../saved_models", exist_ok=True)

model.save("../saved_models/toxicity_model.keras")


Epoch 1/7
939/939 ━━━━━━━━━━━━━━━━━━━━ 133s 137ms/step - accuracy: 0.9942 - loss: 0.0543 - val_accuracy: 0.9940 - val_loss: 0.0527
Epoch 2/7
939/939 ━━━━━━━━━━━━━━━━━━━━ 144s 139ms/step - accuracy: 0.9943 - loss: 0.0508 - val_accuracy: 0.9937 - val_loss: 0.0529
Epoch 3/7
939/939 ━━━━━━━━━━━━━━━━━━━━ 127s 135ms/step - accuracy: 0.9940 - loss: 0.0485 - val_accuracy: 0.9939 - val_loss: 0.0542
Epoch 4/7
939/939 ━━━━━━━━━━━━━━━━━━━━ 127s 135ms/step - accuracy: 0.9937 - loss: 0.0461 - val_accuracy: 0.9940 - val_loss: 0.0549
Epoch 5/7
939/939 ━━━━━━━━━━━━━━━━━━━━ 125s 133ms/step - accuracy: 0.9932 - loss: 0.0437 - val_accuracy: 0.9939 - val_loss: 0.0557
Epoch 6/7
939/939 ━━━━━━━━━━━━━━━━━━━━ 132s 141ms/step - accuracy: 0.9917 - loss: 0.0411 - val_accuracy: 0.9939 - val_loss: 0.0584
Epoch 7/7
939/939 ━━━━━━━━━━━━━━━━━━━━ 130s 139ms/step - accuracy: 0.9915 - loss: 0.0388 - val_accuracy: 0.9939 - val_loss: 0.0599


In [30]:
from tensorflow.keras.models import load_model

model = load_model("../saved_models/toxicity_model.keras")

sample = ["You are stupid"]

sample_clean = [clean_text(sample[0])]
sample_pad, _ = tokenize_and_pad(sample_clean)

prediction = model.predict(sample_pad)

print(prediction)



1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 874ms/step
[[1.00755826e-01 7.88216494e-05 1.15790833e-02 4.61035524e-04
  1.02212625e-02 2.85220239e-03]]


In [32]:
import os
os.makedirs("saved_models", exist_ok=True)

model.save("saved_models/toxicity_model.keras")
